In [ ]:
#저장된 bounding box 를 사진과 맞추어 보면서, bounding box위치를 확인함.

In [ ]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd

from PIL import Image
from tqdm.notebook import tqdm
import time

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import multiprocessing as mp
from IPython.display import clear_output

from enum import Enum

In [ ]:
databb_dir = os.path.join(os.getcwd(), '..', '..', 'databb', 'train', 'images')
#dataimg_dir = os.path.join(os.getcwd(), '..', '..', 'data', 'train', 'images')

In [ ]:
class MaskLabels(int, Enum):
    MASK = 0
    INCORRECT = 1
    NORMAL = 2

_file_names = {
        "mask1": MaskLabels.MASK,
        "mask2": MaskLabels.MASK,
        "mask3": MaskLabels.MASK,
        "mask4": MaskLabels.MASK,
        "mask5": MaskLabels.MASK,
        "incorrect_mask": MaskLabels.INCORRECT,
        "normal": MaskLabels.NORMAL
    }

In [ ]:
# train 의 bounding box

In [ ]:
#check_list = []

processed = 0
visual = True
manualcheck = False

profiles = os.listdir(databb_dir) # check bounding box dir
for profile in profiles:
    print(processed)

    dataimg_dir = databb_dir.replace("databb", "data")
    img_folder = os.path.join(dataimg_dir, profile)
    print(img_folder)
    
    imgs = []
    bboxpatch = []
    bbox = np.zeros((7, 4)) 
    ages = []
    avgbbox = []
    avgcoord = np.zeros(4)
    cnt = 0
    for file_name in os.listdir(img_folder):
        _file_name, ext = os.path.splitext(file_name)
        if _file_name not in _file_names:
            continue
        elif ext != ".jpg": # jpg 파일만 처리
            continue
    
        img_path = os.path.join(dataimg_dir, profile, file_name)  
        imgs.append(np.array(Image.open(img_path)))
       
        bb_path = os.path.join(databb_dir, profile, _file_name + ".txt")  
        bboxfile = open(bb_path, 'r')
    
        if bboxfile is not None:
            bboxcoord = bboxfile.read().split(',', maxsplit=4)
            for i in range(4):
                bbox[cnt][i] = int(bboxcoord[i])
                avgcoord[i] += int(bboxcoord[i])
            bboxfile.close()
            if visual:
                bboxpatch.append(patches.Rectangle((bbox[cnt][0], bbox[cnt][1]), bbox[cnt][2], bbox[cnt][3], linewidth=3, edgecolor='b', facecolor='none'))
            cnt = cnt + 1
        else:
            bbox = np.zeros(4)  
    
    
        processed = processed + 1
    
    if visual and len(bboxpatch) >= 2:
        avgcoord/=len(bboxpatch)
        for i in range(len(bboxpatch)):
            avgbbox.append(patches.Rectangle((avgcoord[0], avgcoord[1]), avgcoord[2], avgcoord[3], linewidth=3, edgecolor='r', facecolor='none'))

        # bbox 잘못된것 검사하는 규칙
        margin = 0.3
        for i in range(len(bboxpatch)):
            # Width
            if (((avgcoord[2] * (1.0 + margin)) < bbox[i][2]) or ((avgcoord[2] * margin) > bbox[i][2])):
                manualcheck = True
                clear_output()
                print(i)
                print((avgcoord[2] * margin) , (avgcoord[2] * (1.0 + margin)), bbox[i][2])
                break
            # Height
            elif (((avgcoord[3] * (1.0 + margin)) < bbox[i][3]) or ((avgcoord[3] * margin) > bbox[i][3])):
                manualcheck = True
                clear_output()
                print(i)
                print((avgcoord[3] * margin) , (avgcoord[3] * (1.0 + margin)), bbox[i][3])
                break
            elif (bbox[i][0] == 0):
                manualcheck = True
                clear_output()
                print(i)
                break
        
        # 수작업
        if manualcheck:
            print("AVG : %d, %d, %d, %d" % (int(avgcoord[0]), int(avgcoord[1]), int(avgcoord[2]), int(avgcoord[3])))
            
            
            # 화면에 그린다
            print(img_folder)
            n_rows, n_cols = 2, 4
            fig, axes = plt.subplots(n_rows, n_cols, sharex=True, sharey=True, figsize=(15, 10))
            for i in range(len(bboxpatch)):
                print("%d : %d, %d, %d, %d" % (i, bbox[i][0], int(bbox[i][1]), int(bbox[i][2]), int(bbox[i][3])))
                axes[i//(n_rows+2)][i%n_cols].imshow(imgs[i])
                axes[i//(n_rows+2)][i%n_cols].add_patch(bboxpatch[i])
                axes[i//(n_rows+2)][i%n_cols].add_patch(avgbbox[i]) # avg bbox

            plt.tight_layout()
            plt.show()
        
            check = True
            while check:
                check_ = input('1 -> pass, 0 -> check : ')
                if check_ == '1':
                    check = False
                elif check_ == '0':
               #     check_list.append(img_folder)
                    check = False
                else:
                    pass
            manualcheck = False
        

In [ ]:
#databb_dir = os.path.join(os.getcwd(), '..', '..', 'databb', 'eval', 'images')
dataimg_dir = os.path.join(os.getcwd(), '..', '..', 'data', 'eval', 'images')

In [ ]:
# eval 의 bounding box

In [ ]:
processed = 0
visual = True
manualcheck = False

check_list = []

img_path = dataimg_dir
cnt = 0
imgs = []
bboxpatch = []
bbox = np.zeros((7, 4)) 
ages = []
avgbbox = []
avgcoord = np.zeros(4)
for file_name in os.listdir(img_path):
    if file_name.find("._") != -1:
        continue
    elif ext != ".jpg": # jpg 파일만 처리
        continue

    #img_path = os.path.join(dataimg_dir, profile, file_name)  
    imgs = (np.array(Image.open( os.path.join(img_path, file_name)  )))

    bb_path = img_path.replace("data", "databb")
    
    bb_file = os.path.join(bb_path, file_name.replace("jpg", "txt"))
    if (os.path.isfile(bb_file)):
        bboxfile = open(bb_file, 'r')

        if bboxfile is not None:
            bboxcoord = bboxfile.read().split(',', maxsplit=4)
            for i in range(4):
                bbox[0][i] = int(bboxcoord[i])
                avgcoord[i] = int(bboxcoord[i])
            bboxfile.close()
            if visual:
                bboxpatch = (patches.Rectangle((bbox[cnt][0], bbox[cnt][1]), bbox[cnt][2], bbox[cnt][3], linewidth=3, edgecolor='b', facecolor='none'))
        #    cnt = cnt + 1
        else:
            bbox = np.zeros(4)  


        processed = processed + 1

        if visual:# and len(bboxpatch) >= 1:
            # bbox 잘못된것 검사하는 규칙
            if (bbox[0][0] == 0):
                i = 0
                manualcheck = True
                clear_output()
                print(i)

                # 수작업
                if manualcheck:

                    # 화면에 그린다
                    print(file_name)
                    n_rows, n_cols = 2,2
                    fig, axes = plt.subplots(n_rows, n_cols, sharex=True, sharey=True, figsize=(15, 10))

                    print("%d : %d, %d, %d, %d" % (i, bbox[i][0], int(bbox[i][1]), int(bbox[i][2]), int(bbox[i][3])))
                    axes[0][0].imshow(imgs)
                    axes[0][0].add_patch(bboxpatch)

                    plt.tight_layout()
                    plt.show()

                    #os.remove(bb_file)
                    
                    check = True
                    while check:
                        check_ = input('1 -> pass, 0 -> check : ')
                        if check_ == '1':
                            check = False
                        elif check_ == '0':
                            check_list.append(img_folder)
                            check = False
                        else:
                            pass
                    manualcheck = False
